Import relevant libraries

In [17]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from torchvision import datasets, transforms

Load Dataset

In [18]:
# Define a transform to convert images to tensors
transform = transforms.ToTensor()

# Load the MNIST dataset
mnist_train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Create data loaders to load the data in batches
data_loader = torch.utils.data.DataLoader(dataset=mnist_train_data, batch_size=64, shuffle=True)
data_loader_test = torch.utils.data.DataLoader(dataset=mnist_test_data, batch_size=10000, shuffle=False)

Define the Autoencoder

In [19]:

# This is the definition of the Autoencoder model using a neural network with encoder and decoder modules.
class Autoencoder400D(nn.Module):
    def __init__(self):
        super(Autoencoder400D, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(784, 700),
            nn.ReLU(),
            nn.Linear(700, 500),
            nn.ReLU(),
            nn.Linear(500, 400)
        )

        self.decoder = nn.Sequential(
            nn.Linear(400, 500),
            nn.ReLU(),
            nn.Linear(500, 700),
            nn.ReLU(),
            nn.Linear(700, 784),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

Define the relevant model parameters

In [20]:
# These lines define the loss function (MSE loss), instantiate the Autoencoder model
# learning rate of 1e-3 and L2 regularization with a weight decay of 1e-5.
criterion = nn.MSELoss()
model = Autoencoder400D()
optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-10, momentum=0.9)

train the model

In [21]:
# train the model over n epochs
num_epochs = 100
encPics = []
labels = []
for epoch in range(num_epochs):
    for (img, _) in data_loader:
        img = img.reshape(-1, 28 * 28)
        recon = model(img)
        loss = criterion(recon, img)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch:{epoch + 1}, Loss:{loss.item():.4f}')

Epoch:1, Loss:0.0637
Epoch:2, Loss:0.0624
Epoch:3, Loss:0.0688
Epoch:4, Loss:0.0591
Epoch:5, Loss:0.0577
Epoch:6, Loss:0.0500
Epoch:7, Loss:0.0488
Epoch:8, Loss:0.0406
Epoch:9, Loss:0.0389
Epoch:10, Loss:0.0326
Epoch:11, Loss:0.0323
Epoch:12, Loss:0.0285
Epoch:13, Loss:0.0270
Epoch:14, Loss:0.0255
Epoch:15, Loss:0.0261
Epoch:16, Loss:0.0223
Epoch:17, Loss:0.0256
Epoch:18, Loss:0.0220
Epoch:19, Loss:0.0201
Epoch:20, Loss:0.0226
Epoch:21, Loss:0.0187
Epoch:22, Loss:0.0218
Epoch:23, Loss:0.0190
Epoch:24, Loss:0.0177
Epoch:25, Loss:0.0155
Epoch:26, Loss:0.0163
Epoch:27, Loss:0.0166
Epoch:28, Loss:0.0144
Epoch:29, Loss:0.0157
Epoch:30, Loss:0.0152
Epoch:31, Loss:0.0148
Epoch:32, Loss:0.0127
Epoch:33, Loss:0.0155
Epoch:34, Loss:0.0121
Epoch:35, Loss:0.0157
Epoch:36, Loss:0.0131
Epoch:37, Loss:0.0153
Epoch:38, Loss:0.0132
Epoch:39, Loss:0.0131
Epoch:40, Loss:0.0132
Epoch:41, Loss:0.0126
Epoch:42, Loss:0.0117
Epoch:43, Loss:0.0134
Epoch:44, Loss:0.0123
Epoch:45, Loss:0.0123
Epoch:46, Loss:0.01

Determine the accuracy of the Random Forest classifier on the reduced data

In [22]:
for (img, l) in data_loader_test:
    item = img.reshape(-1, 28 * 28)
    enc = model.encoder(item)
    enc = enc.detach().numpy()
    l = l.detach().numpy()
    encPics.append(enc)
    labels.append(l)

rf = RandomForestClassifier(n_estimators=266, max_depth=11)
encPics = np.reshape(encPics, (-1, 400))
labels = np.reshape(labels, (-1, 1))
labels = labels.ravel()
rf.fit(encPics, labels)
y_pred_1 = rf.predict(encPics)

accscore = accuracy_score(labels, y_pred_1, normalize=True)
print(accscore)

0.9966
